# Trajectories from emulsion droplets

Load the segmented droplet table (x, y, z positions, radius `r`, frame `t`, droplet `id`) and convert it into a `TrajectorySet` with the radius stored as a per-frame feature.


In [ ]:
import pandas as pd
from trajkit import TrajectorySet

data_path = "/Users/mehdi/HuggingFace/trajkitData/data/csv/EmulsionSGM_Traj.csv"

df = pd.read_csv(data_path)
if df.columns[0].startswith("Unnamed"):
    df = df.drop(columns=df.columns[0])

# Use an explicit frame column instead of the raw `t` name from the file.
df = df.rename(columns={"t": "frame"})
df.head()


Build a `TrajectorySet` from the per-frame table. The radius column is passed via `frame_feature_cols` so it is attached to each frame in `Trajectory.frame_features`.


In [ ]:
frame_rate_hz = 1.0  # update to your acquisition rate (frames per second)

trajset = TrajectorySet.from_dataframe(
    df,
    dataset_id="emulsion_sgm",
    track_id_col="id",
    position_cols=["x", "y", "z"],
    frame_col="frame",
    frame_rate_hz=frame_rate_hz,
    frame_feature_cols=["r"],
    units={"t": "frame", "x": "px"},
    meta={"source": "Emulsion stable distribution"},
)

trajset.summary_table().head()


Inspect the stored frame-level metadata. Radius values live in `Trajectory.frame_features["r"]`, and `to_frame_dataframe()` exposes them with an `ff_` prefix for analysis or plotting.


In [ ]:
example_id = trajset.ids()[0]
tr = trajset.get(example_id)

print(f"Frame-level feature keys: {list(tr.frame_features.keys())}")
print(tr.frame_features["r"][:5])

tr.to_frame_dataframe().head()


In [ ]:
import pandas as pd
from trajkit.viz.three_d import TrajectoryFrameSpec, ensure_dataframe, animate_3d_plotly
# Load your droplet table (id, t, x, y, z, r)
data_path = "/Users/mehdi/HuggingFace/trajkitData/data/csv/EmulsionSGM_Traj.csv"
df = pd.read_csv(data_path)
if df.columns[0].startswith("Unnamed"):
    df = df.drop(columns=df.columns[0])
df = df.rename(columns={"t": "t"})  # adjust if you renamed t→frame elsewhere

# Build figure
spec = TrajectoryFrameSpec(id_col="id", t_col="t", x_col="x", y_col="y", z_col="z", r_col="r")
df = ensure_dataframe(df, spec)
fig = animate_3d_plotly(
    df,
    spec=spec,
    marker_scale=1,
    axis_padding=0.05,
    camera_eye=(1.8, 1.2, 1.4),  # adjust default view
)
fig.show()



In [ ]:
from trajkit.viz.three_d import TrajectoryFrameSpec, ensure_dataframe, animate_3d_mpl

spec = TrajectoryFrameSpec()
df = ensure_dataframe(df, spec)

anim = animate_3d_mpl(df, spec=spec, marker_scale=1.0, fps=20, output=None)
# or save: animate_3d_mpl(df, spec=spec, output="droplets.mp4")


In [ ]:
from trajkit.viz.three_d import animate_3d_pyvista
import pandas as pd
data_path = "/Users/mehdi/HuggingFace/trajkitData/data/csv/EmulsionSGM_Traj.csv"
df = pd.read_csv(data_path)
animate_3d_pyvista(df, output="droplets.gif")


In [ ]:
import sys, subprocess, json, os, sys
print(sys.executable)
print(sys.path[:5])
subprocess.run([sys.executable, "-m", "pip", "show", "pyvista"])


In [ ]:
from trajkit.viz.three_d import napari_view_tracks
napari_view_tracks(df)
